### How to scrape geospatial data from Google Maps (because the API limits results)

In [1]:
import re
import pandas as pd
from glob import glob

In [2]:
chi_sb = glob('./data/ChicagoResults/sb*')
chi_nsb = glob('./data/ChicagoResults/nsb*')

In [3]:
def loadHTML(f):
    conn = open(f,encoding='latin-1')
    html = conn.read()
    conn.close()
    html = re.sub('\n','',html)
    html = re.sub('=','',html)
    return html

def scraper(html,sbInd):
    if sbInd == True:
        patt = re.compile('([1-9]{2}\.\d{6,}),(-\d{2,3}\.\d{6,}).{100,900}\"([\w ,-]{,20}Starbucks[\w -]{,30}), ((\d{,6} )?[,\w ]{,100}), ([\w ]{,10}), ([A-Z]{2}) (\d{5}(\-\d{4}){,1})')
        inds = [0,1,2,3,5,6,7]
    elif sbInd == False:
        patt = re.compile('([1-9]{2}\.\d{6,}),(-\d{2,3}\.\d{6,}).{100,900}\"([\w ,-]{,50}([Cc][oa]ff*ee*[\w ,-]{,50})), ((\d{,6} )?[,\w ]{,100}), ([\w]{,10}), ([A-Z]{2}) (\d{5}(\-\d{4}){,1})')
        inds = [0,1,2,4,6,7,8]
    results = pd.DataFrame(patt.findall(html))

    #results = results[inds]
    #results.columns = ['latitide','longitude','name','address','city','state','zipcode']
    return results

def mineHTML(search,sbInd):
    # true for starbucks, false for not starbucks
    dfList = []
    for htmlUrl in search:
        html = loadHTML(htmlUrl)
        scrapedDF = scraper(html,sbInd)
        dfList.append(scrapedDF)
    
    return pd.concat(dfList)

In [4]:
#testHTML = loadHTML(chi_nsb[0])
x = mineHTML(chi_sb,True)
x = x.drop_duplicates(subset=[4])
x
#x[[0,1,2,4,6,7,8]]

,0,1,2,3,4,5,6,7,8
0,41.910350199999996,-87.67763479999999,Starbucks,1588 N Milwaukee Ave,1588,Chicago,IL,60622,
1,41.903143,-87.670091,Starbucks,1701 W Division St,1701,Chicago,IL,60622,
2,41.9174934,-87.6802943,Starbucks,2101 W Armitage Ave,2101,Chicago,IL,60647,
3,41.9106082,-87.6535305,Starbucks,1001 W North Ave e,1001,Chicago,IL,60642,
4,41.9254843,-87.6723274,Starbucks,1730 W Fullerton Ave,1730,Chicago,IL,60614,
6,41.928014999999995,-87.6821774,Starbucks,2566 N Elston Ave,2566,Chicago,IL,60647,
7,41.8916454,-87.6589048,Starbucks,520 N Ogden Ave,520,Chicago,IL,60642,
8,41.9325663,-87.6709946,Starbucks,1700 W Diversey Pkwy,1700,Chicago,IL,60614,
9,41.9279784,-87.69738199999999,Starbucks,2543 N California Ave,2543,Chicago,IL,60647,
10,41.937579,-87.665959,Starbucks,3045 N Greenview Ave,3045,Chicago,IL,60657,


In [5]:
# patt = re.compile('([1-9]{2}\.\d{6,}),(-\d{2,3}\.\d{6,}).{100,900}\"([\w ,-]{,50}([Cc][oa]ff*ee*[\w ,-]{,50})), ((\d{,6} )?[,\w ]{,100}), ([\w]{,10}), ([A-Z]{2}) (\d{5}(\-\d{4}){,1})')
# res = re.findall('([1-9]{2}\.\d{6,}),(-\d{2,3}\.\d{6,}).{100,900}\"([\w ,-]{,50}([Cc][oa]ff*ee*[\w ,-]{,50})), ((\d{,6} )?[,\w ]{,100}), ([\w]{,10}), ([A-Z]{2}) (\d{5}(\-\d{4}){,1})',testHTML)
# print(len(res))
# res